In [9]:
import requests
import pandas as pd

In [6]:
r = requests.get('https://covid.ourworldindata.org/data/owid-covid-data.json')

In [182]:
df = pd.DataFrame.from_dict(r.json())
df.head(5)

,AFG,ALB,DZA,AND,AGO,AIA,ATG,ARG,ARM,ABW,...,URY,UZB,VAT,VEN,VNM,ESH,YEM,ZMB,ZWE,OWID_WRL
continent,Asia,Europe,Africa,Europe,Africa,North America,North America,South America,Asia,North America,...,South America,Asia,Europe,South America,Asia,Africa,Asia,Africa,Africa,NaN
location,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Western Sahara,Yemen,Zambia,Zimbabwe,World
population,3.89283e+07,2.8778e+06,4.3851e+07,77265,3.28663e+07,15002,97928,4.51958e+07,2.96323e+06,106766,...,3.47373e+06,3.34692e+07,809,2.84359e+07,9.73386e+07,597330,2.9826e+07,1.8384e+07,1.48629e+07,7.7948e+09
population_density,54.422,104.871,17.348,163.755,23.89,NaN,231.845,16.177,102.931,584.8,...,19.751,76.134,NaN,36.253,308.127,NaN,53.508,22.995,42.729,58.045
median_age,18.6,38,29.1,NaN,16.8,NaN,32.1,31.9,35.7,41.2,...,35.6,28.2,NaN,29,32.6,28.4,20.3,17.7,19.6,30.9


We want rows to be countries not properties so we need to transpose

In [197]:
df2 = df.transpose()
df2.head()

,continent,location,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,data,extreme_poverty,female_smokers,male_smokers
AFG,Asia,Afghanistan,3.89283e+07,54.422,18.6,2.581,1.337,1803.99,597.029,9.59,37.746,0.5,64.83,0.498,"[{'date': '2019-12-31', 'new_cases': 0.0, 'new...",NaN,NaN,NaN
ALB,Europe,Albania,2.8778e+06,104.871,38,13.188,8.643,11803.4,304.195,10.08,NaN,2.89,78.57,0.785,"[{'date': '2020-03-09', 'total_cases': 2.0, 'n...",1.1,7.1,51.2
DZA,Africa,Algeria,4.3851e+07,17.348,29.1,6.211,3.857,13913.8,278.364,6.73,83.741,1.9,76.88,0.754,"[{'date': '2019-12-31', 'new_cases': 0.0, 'new...",0.5,0.7,30.4
AND,Europe,Andorra,77265,163.755,NaN,NaN,NaN,NaN,109.135,7.97,NaN,NaN,83.73,0.858,"[{'date': '2020-03-03', 'total_cases': 1.0, 'n...",NaN,29,37.8
AGO,Africa,Angola,3.28663e+07,23.89,16.8,2.405,1.362,5819.49,276.045,3.94,26.664,NaN,61.15,0.581,"[{'date': '2020-03-22', 'total_cases': 2.0, 'n...",NaN,NaN,NaN


Each entry in the *data* column is actually a list of dicts. Each of these lists is a distinct timeseries dataset. We need to unpack this data. 

First we can explode the field (this creates a row for each list item).

In [198]:
timeseries_packed = df2.explode('data')['data']
timeseries_packed

AFG         {'date': '2019-12-31', 'new_cases': 0.0, 'new_...
AFG         {'date': '2020-01-01', 'new_cases': 0.0, 'new_...
AFG         {'date': '2020-01-02', 'new_cases': 0.0, 'new_...
AFG         {'date': '2020-01-03', 'new_cases': 0.0, 'new_...
AFG         {'date': '2020-01-04', 'new_cases': 0.0, 'new_...
                                  ...                        
OWID_WRL    {'date': '2020-10-20', 'total_cases': 40517993...
OWID_WRL    {'date': '2020-10-21', 'total_cases': 40909132...
OWID_WRL    {'date': '2020-10-22', 'total_cases': 41353615...
OWID_WRL    {'date': '2020-10-23', 'total_cases': 41810959...
OWID_WRL    {'date': '2020-10-24', 'total_cases': 42279076...
Name: data, Length: 51923, dtype: object

Then we can expand each of these dicts into a set of fields. `Series.apply` just runs every value through a function of your choice. By converting each value in a `Series` to a `Series`, we end up with a `Series` of `Series`, which is a 2-dimensional dataset, or a `DataFrame`.

In [199]:
timeseries_unpacked = timeseries_packed.apply(pd.Series)
timeseries_unpacked

,date,new_cases,new_deaths,new_cases_per_million,new_deaths_per_million,stringency_index,new_cases_smoothed,new_deaths_smoothed,new_cases_smoothed_per_million,new_deaths_smoothed_per_million,...,total_deaths_per_million,total_tests,total_tests_per_thousand,tests_units,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,new_tests,new_tests_per_thousand
AFG,2019-12-31,0.0,0.0,0.000,0.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,2020-01-01,0.0,0.0,0.000,0.000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,2020-01-02,0.0,0.0,0.000,0.000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,2020-01-03,0.0,0.0,0.000,0.000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFG,2020-01-04,0.0,0.0,0.000,0.000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OWID_WRL,2020-10-20,348284.0,4670.0,44.682,0.599,NaN,371120.286,5397.143,47.611,0.692,...,143.630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OWID_WRL,2020-10-21,391139.0,6405.0,50.179,0.822,NaN,379757.429,5522.143,48.719,0.708,...,144.452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OWID_WRL,2020-10-22,444483.0,6740.0,57.023,0.865,NaN,387778.429,5622.571,49.748,0.721,...,145.317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OWID_WRL,2020-10-23,457344.0,6349.0,58.673,0.815,NaN,395632.714,5656.571,50.756,0.726,...,146.131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we have a dataset that has a single row for each country/date pair, with the timeseries data properties expanded into columns correctly. 

We could leave this here and use this dataset by itself (or in conjunction with the first). 

There might be some calculations requiring data from both, in which case it might help to join the datasets so we have both the country properties and the timeseries data in the same `DataFrame`. We could also drop the old *data* columns as we don't need it now that we have expanded it.

In [208]:
joined_df = df2.drop('data', axis='columns').join(timeseries_unpacked)
joined_df

,continent,location,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,...,total_deaths_per_million,total_tests,total_tests_per_thousand,tests_units,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,new_tests,new_tests_per_thousand
ABW,North America,Aruba,106766,584.8,41.2,13.085,7.452,35973.8,NaN,11.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW,North America,Aruba,106766,584.8,41.2,13.085,7.452,35973.8,NaN,11.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW,North America,Aruba,106766,584.8,41.2,13.085,7.452,35973.8,NaN,11.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW,North America,Aruba,106766,584.8,41.2,13.085,7.452,35973.8,NaN,11.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW,North America,Aruba,106766,584.8,41.2,13.085,7.452,35973.8,NaN,11.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE,Africa,Zimbabwe,1.48629e+07,42.729,19.6,2.822,1.882,1899.78,307.846,1.82,...,15.609,132239.0,8.897,tests performed,639.0,0.043,32.413,0.031,791.0,0.053
ZWE,Africa,Zimbabwe,1.48629e+07,42.729,19.6,2.822,1.882,1899.78,307.846,1.82,...,15.677,133044.0,8.951,tests performed,666.0,0.045,30.874,0.032,805.0,0.054
ZWE,Africa,Zimbabwe,1.48629e+07,42.729,19.6,2.822,1.882,1899.78,307.846,1.82,...,15.878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZWE,Africa,Zimbabwe,1.48629e+07,42.729,19.6,2.822,1.882,1899.78,307.846,1.82,...,15.878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Because each row represents a country/date pair, you can see that the country properties are replicated for each row. This isn't a problem.

We can make the dataset a little clearer by using a multi-dimensional index (country-date)


In [213]:
final_df = joined_df.set_index('date', append=True, drop=False)
final_df

continent  location   population population_density  \
    date                                                                  
ABW 2020-03-13  North America     Aruba       106766              584.8   
    2020-03-19  North America     Aruba       106766              584.8   
    2020-03-20  North America     Aruba       106766              584.8   
    2020-03-21  North America     Aruba       106766              584.8   
    2020-03-22  North America     Aruba       106766              584.8   
...                       ...       ...          ...                ...   
ZWE 2020-10-20         Africa  Zimbabwe  1.48629e+07             42.729   
    2020-10-21         Africa  Zimbabwe  1.48629e+07             42.729   
    2020-10-22         Africa  Zimbabwe  1.48629e+07             42.729   
    2020-10-23         Africa  Zimbabwe  1.48629e+07             42.729   
    2020-10-24         Africa  Zimbabwe  1.48629e+07             42.729   

               median_age aged_65_older aged_70_older gdp_per_capita  \
    date                                                               
ABW 2020-03-13       41.2        13.085         7.452        35973.8   
    2020-03-19       41.2        13.085         7.452        35973.8   
    2020-03-20       41.2        13.085         7.452        35973.8   
    2020-03-21       41.2        13.085         7.452        35973.8   
    2020-03-22       41.2        13.085         7.452        35973.8   
...                   ...           ...           ...            ...   
ZWE 2020-10-20       19.6         2.822         1.882        1899.78   
    2020-10-21       19.6         2.822         1.882        1899.78   
    2020-10-22       19.6         2.822         1.882        1899.78   
    2020-10-23       19.6         2.822         1.882        1899.78   
    2020-10-24       19.6         2.822         1.882        1899.78   

               cardiovasc_death_rate diabetes_prevalence  ...  \
    date                                                  ...   
ABW 2020-03-13                   NaN               11.62  ...   
    2020-03-19                   NaN               11.62  ...   
    2020-03-20                   NaN               11.62  ...   
    2020-03-21                   NaN               11.62  ...   
    2020-03-22                   NaN               11.62  ...   
...                              ...                 ...  ...   
ZWE 2020-10-20               307.846                1.82  ...   
    2020-10-21               307.846                1.82  ...   
    2020-10-22               307.846                1.82  ...   
    2020-10-23               307.846                1.82  ...   
    2020-10-24               307.846                1.82  ...   

               total_deaths_per_million total_tests total_tests_per_thousand  \
    date                                                                       
ABW 2020-03-13                      NaN         NaN                      NaN   
    2020-03-19                      NaN         NaN                      NaN   
    2020-03-20                      NaN         NaN                      NaN   
    2020-03-21                      NaN         NaN                      NaN   
    2020-03-22                      NaN         NaN                      NaN   
...                                 ...         ...                      ...   
ZWE 2020-10-20                   15.609    132239.0                    8.897   
    2020-10-21                   15.677    133044.0                    8.951   
    2020-10-22                   15.878         NaN                      NaN   
    2020-10-23                   15.878         NaN                      NaN   
    2020-10-24                   15.878         NaN                      NaN   

                    tests_units new_tests_smoothed  \
    date                                             
ABW 2020-03-13              NaN                NaN   
    2020-03-19              NaN                NaN   
    